In [1]:
#We load in the specific data needed
import polars as pl
import numpy as np
import os
from pathlib import Path
import pandas as pd
from datetime import date
import datetime
import hvplot.polars
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.interpolate import bisplrep, bisplev
from datetime import timedelta
from patsy import dmatrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [2]:
tot_data_train_c = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/tech_data/data_set_train_val_tech_scaled_c.parquet')
tot_data_train_p = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/tech_data/data_set_train_val_tech_scaled_p.parquet')


# firm_data = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/Firm_variables/daily_firm_data_median_new.parquet')


columns_to_drop = ['moneyness_squared_mean', 'moneyness_group', 'tau_squared_mean', 'moneyness_tau_mean', 'tau_squared_std', 'moneyness_tau_std', 'T_std', 'impl_volatility_std', 'moneyness_squared_std']

tot_data_train_c = tot_data_train_c.drop(columns=columns_to_drop, errors='ignore')
tot_data_train_p = tot_data_train_p.drop(columns=columns_to_drop, errors='ignore')


In [3]:
tot_data_train_c

date  moneyness_mean  moneyness_std  impl_volatility_mean  \
10   2019-01-02       -1.559693      -0.899796              0.487425   
11   2019-01-02       -1.238194       0.280259              0.480363   
12   2019-01-02       -0.862361      -0.483196              0.484917   
13   2019-01-02       -0.517492       0.095575              0.495387   
14   2019-01-02       -0.154267      -0.016268              0.485254   
...         ...             ...            ...                   ...   
5035 2020-12-31        0.116888       0.118652              0.376182   
5036 2020-12-31        0.514576       0.449081              0.378704   
5037 2020-12-31        0.858547      -6.472006              0.389960   
5038 2020-12-31        1.192630       1.742726              0.378551   
5039 2020-12-31        1.564968      -0.597320              0.395207   

        T_mean  volume_option_mean  volume_option_std  spread_option_mean  \
10   -0.000623           -0.892034          -0.775850            1.931987   
11   -0.000623           -0.672816          -0.593749            0.841789   
12   -0.000623           -0.541080          -0.462838            0.101784   
13   -0.000623           -0.526974          -0.629729           -0.087657   
14   -0.000623           -0.392774          -0.166903           -0.229368   
...        ...                 ...                ...                 ...   
5035 -1.520941            4.338761           3.741729           -0.552380   
5036 -1.520941            4.850644           6.542937           -0.596780   
5037 -1.520941            2.451356           2.170486           -0.626380   
5038 -1.520941            0.689877           0.521470           -0.612320   
5039 -1.520941           -0.365531          -0.576701           -0.621200   

      spread_option_std  prc_option_mean  ...  cp_flag   FF_rate  gold_price  \
10             2.002133         1.143992  ...        C  1.126349   -1.357150   
11             1.048057         0.555735  ...        C  1.126349   -1.357150   
12             0.096188         0.303572  ...        C  1.126349   -1.357150   
13            -0.114131         0.039180  ...        C  1.126349   -1.357150   
14            -0.170793        -0.190332  ...        C  1.126349   -1.357150   
...                 ...              ...  ...      ...       ...         ...   
5035          -0.544018        -0.497352  ...        C -1.162816    1.436988   
5036          -0.608031        -0.545443  ...        C -1.162816    1.436988   
5037          -0.608329        -0.584979  ...        C -1.162816    1.436988   
5038          -0.601425        -0.585851  ...        C -1.162816    1.436988   
5039          -0.622706        -0.597199  ...        C -1.162816    1.436988   

      reces_indi   10Y_RIR   1Y_bond   2Y_bond  OPEN_vix  CLOSE_vix  hi-lo_vix  
10     -0.709812  1.566975  1.374613  1.472822  0.677893   0.478375  -0.134139  
11     -0.709812  1.566975  1.374613  1.472822  0.677893   0.478375  -0.134139  
12     -0.709812  1.566975  1.374613  1.472822  0.677893   0.478375  -0.134139  
13     -0.709812  1.566975  1.374613  1.472822  0.677893   0.478375  -0.134139  
14     -0.709812  1.566975  1.374613  1.472822  0.677893   0.478375  -0.134139  
...          ...       ...       ...       ...       ...        ...        ...  
5035    0.268241 -0.801314 -1.215175 -1.208572 -0.056028  -0.033654  -0.552581  
5036    0.268241 -0.801314 -1.215175 -1.208572 -0.056028  -0.033654  -0.552581  
5037    0.268241 -0.801314 -1.215175 -1.208572 -0.056028  -0.033654  -0.552581  
5038    0.268241 -0.801314 -1.215175 -1.208572 -0.056028  -0.033654  -0.552581  
5039    0.268241 -0.801314 -1.215175 -1.208572 -0.056028  -0.033654  -0.552581  

[5030 rows x 51 columns]

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

def feature_permutation_importance(train_data, n_estimators=100, random_state=42, verbose=True):
    """
    Perform permutation importance on the training data to identify the top features.
    
    Parameters:
    - train_data (pd.DataFrame): The training dataset.
    - n_estimators (int): Number of estimators (trees) for the RandomForestRegressor.
    - random_state (int): Seed for reproducibility.
    - verbose (bool): If True, prints progress information.
    
    Returns:
    - feature_importances (pd.DataFrame): A dataframe containing feature names and their importance.
    """
    # Prepare the train data
    X_train = train_data.drop(columns=['impl_volatility_mean', 'date', 'cp_flag'])  # Drop unwanted columns
    y_train = train_data['impl_volatility_mean']  # Target variable (implied volatility)

    # Standardize the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Train a RandomForestRegressor model
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    
    if verbose:
        print(f"Training RandomForest model with {n_estimators} trees...")

    rf_model.fit(X_train_scaled, y_train)

    if verbose:
        print("Performing permutation importance...")

    # Perform permutation importance on the training data
    perm_importance = permutation_importance(rf_model, X_train_scaled, y_train, n_repeats=10, random_state=random_state, scoring='r2')

    # Create a DataFrame for feature importances
    feature_importances = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': perm_importance.importances_mean,
        'Importance_std': perm_importance.importances_std
    })

    # Sort by importance
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False).reset_index(drop=True)

    return feature_importances


def plot_elbow_curve(feature_importances):
    """
    Plot an elbow curve to determine how many features to select based on permutation importance.
    
    Parameters:
    - feature_importances (pd.DataFrame): A dataframe containing feature names and their importance.
    """
    # Cumulative importance to capture the "elbow"
    cumulative_importance = np.cumsum(feature_importances['Importance'])

    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_importance) + 1), cumulative_importance, marker='o', linestyle='--')
    plt.title('Elbow Plot of Feature Importance')
    plt.xlabel('Number of Features')
    plt.ylabel('Cumulative Importance')
    plt.grid(True)
    plt.show()


def select_top_features(feature_importances, threshold=0.85):
    """
    Select top features based on a cumulative importance threshold.
    
    Parameters:
    - feature_importances (pd.DataFrame): A dataframe containing feature names and their importance.
    - threshold (float): Cumulative importance threshold (e.g., 0.85 for 85%).
    
    Returns:
    - selected_features (list): A list of top features to use in the model.
    """
    cumulative_importance = np.cumsum(feature_importances['Importance'])
    
    # Find how many features explain the threshold (e.g., 85%) of importance
    num_features = np.argmax(cumulative_importance >= threshold) + 1
    
    selected_features = feature_importances['Feature'].iloc[:num_features].tolist()
    print(f"\nSelected {num_features} features based on an {threshold*100}% importance threshold.")
    
    return selected_features


# Example usage for tot_data_train_c
print("Evaluating features for Call options...")
feature_importances_call = feature_permutation_importance(tot_data_train_c, verbose=True)

# Display top features and their importance
print("\nFeature importances for Call options:")
print(feature_importances_call)

# Plot the elbow curve to decide on the number of features
plot_elbow_curve(feature_importances_call)

# Select top features (e.g., 85% cumulative importance)
selected_features_call = select_top_features(feature_importances_call, threshold=0.85)

print("\nSelected top features for Call options:")
print(selected_features_call)

# Example usage for tot_data_train_p
print("\nEvaluating features for Put options...")
feature_importances_put = feature_permutation_importance(tot_data_train_p, verbose=True)

# Display top features and their importance
print("\nFeature importances for Put options:")
print(feature_importances_put)

# Plot the elbow curve to decide on the number of features
plot_elbow_curve(feature_importances_put)

# Select top features (e.g., 85% cumulative importance)
selected_features_put = select_top_features(feature_importances_put, threshold=0.85)

print("\nSelected top features for Put options:")
print(selected_features_put)


Evaluating features for Call options...
Training RandomForest model with 100 trees...
Performing permutation importance...

Feature importances for Call options:
                              Feature  Importance  Importance_std
0                             prev_iv    1.423783        0.020639
1                              T_mean    0.195029        0.010874
2                          gold_price    0.053833        0.002348
3                         prev_iv_std    0.036145        0.001470
4                            prev2_iv    0.030608        0.000758
5                          reces_indi    0.018196        0.000578
6     5_day_rolling_return_stock_mean    0.014699        0.000801
7                            OPEN_vix    0.013285        0.000300
8                           hi-lo_vix    0.012512        0.000866
9                        prev2_iv_std    0.010331        0.000238
10                   spread_stock_std    0.009825        0.000297
11                     vol_stock_mean    0.008


Selected 1 features based on an 85.0% importance threshold.

Selected top features for Call options:
['prev_iv']

Evaluating features for Put options...
Training RandomForest model with 100 trees...
Performing permutation importance...

Feature importances for Put options:
                              Feature  Importance  Importance_std
0                             prev_iv    1.358627        0.028301
1                              T_mean    0.198175        0.008539
2                           CLOSE_vix    0.030966        0.001366
3                            prev2_iv    0.027269        0.000621
4                          gold_price    0.026144        0.000924
5                         prev_iv_std    0.016199        0.000644
6                        prev2_iv_std    0.012708        0.000602
7                          reces_indi    0.012534        0.000477
8                    spread_stock_std    0.009870        0.000309
9                      moneyness_mean    0.008566        0.000348


Selected 1 features based on an 85.0% importance threshold.

Selected top features for Put options:
['prev_iv']


In [5]:
feature_importances_call

# BEGIN: Normalize the importance
feature_importances_call['Normalized_Importance'] = feature_importances_call['Importance'] / feature_importances_call['Importance'].sum()
feature_importances_put['Normalized_Importance'] = feature_importances_put['Importance'] / feature_importances_put['Importance'].sum()

# Combine the two and take the average
combined_importances = pd.merge(
    feature_importances_call[['Feature', 'Normalized_Importance']],
    feature_importances_put[['Feature', 'Normalized_Importance']],
    on='Feature',
    suffixes=('_call', '_put')
)

combined_importances['Average_Importance'] = combined_importances[['Normalized_Importance_call', 'Normalized_Importance_put']].mean(axis=1)

# Display the combined importances
combined_importances = combined_importances.sort_values(by='Average_Importance', ascending=False).reset_index(drop=True)
print(combined_importances)

# Display the top 10 features based on combined importance
top_10_combined_importances = combined_importances.head(10)
print("Top 10 features based on combined importance:")
print(top_10_combined_importances[['Feature', 'Average_Importance']])


                              Feature  Normalized_Importance_call  \
0                             prev_iv                    0.721043   
1                              T_mean                    0.098768   
2                          gold_price                    0.027262   
3                            prev2_iv                    0.015501   
4                         prev_iv_std                    0.018305   
5                           CLOSE_vix                    0.004279   
6                          reces_indi                    0.009215   
7                        prev2_iv_std                    0.005232   
8     5_day_rolling_return_stock_mean                    0.007444   
9                    spread_stock_std                    0.004976   
10                           OPEN_vix                    0.006728   
11                          hi-lo_vix                    0.006336   
12     5_day_rolling_return_stock_std                    0.003967   
13                     vol_stock_m

# Top features based on the shorted dataset

In [12]:
# Get the top 6 most important features
top_6_features_c = feature_importances_call['Feature'].head(30).tolist()

# Print the result
print("top_features_c =", top_6_features_c)

top_features_c = ['prev_iv', 'T_mean', 'gold_price', 'prev_iv_std', 'prev2_iv', 'reces_indi', '5_day_rolling_return_stock_mean', 'OPEN_vix', 'hi-lo_vix', 'prev2_iv_std', 'spread_stock_std', 'vol_stock_mean', '10Y_RIR', 'CLOSE_vix', '5_day_rolling_return_stock_std', '1Y_bond', 'hi-lo_stock_mean', 'RET_mean', 'hi-lo_stock_std', 'PRC_std', 'RET_std', 'FF_rate', 'PRC_actual_mean', 'vol_stock_std', '2Y_bond', 'BID_std', 'moneyness_mean', 'ASK_std', 'spread_stock_mean', 'spread_option_std']


In [13]:
# Get the top 9 most important features
top_9_features_p = feature_importances_put['Feature'].head(30).tolist()

# Print the result
print("top_features_p =", top_9_features_p)

top_features_p = ['prev_iv', 'T_mean', 'CLOSE_vix', 'prev2_iv', 'gold_price', 'prev_iv_std', 'prev2_iv_std', 'reces_indi', 'spread_stock_std', 'moneyness_mean', '5_day_rolling_return_stock_std', 'vol_stock_std', 'hi-lo_stock_std', '5_day_rolling_return_stock_mean', 'RET_std', 'vol_stock_mean', 'NVDA', 'spread_stock_mean', 'hi-lo_vix', 'RET_mean', 'OPEN_vix', 'hi-lo_stock_mean', '1Y_bond', 'AMZN', 'PRC_actual_mean', 'PRC_std', '10Y_RIR', 'prc_option_std', 'ASK_std', 'daily_return_indicator_stock_mean']


# Top features based with frim characteristics

In [8]:
tot_data_train_c_tot = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_val_tech_scaled_c.parquet')
tot_data_train_p_tot  = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_val_tech_scaled_p.parquet')

columns_to_drop = ['trading_days_till_exp']

tot_data_train_c = tot_data_train_c_tot.drop(columns=columns_to_drop, errors='ignore')
tot_data_train_p = tot_data_train_p_tot.drop(columns=columns_to_drop, errors='ignore')


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

def feature_permutation_importance(train_data, n_estimators=100, random_state=42, verbose=True):
    """
    Perform permutation importance on the training data to identify the top features.
    
    Parameters:
    - train_data (pd.DataFrame): The training dataset.
    - n_estimators (int): Number of estimators (trees) for the RandomForestRegressor.
    - random_state (int): Seed for reproducibility.
    - verbose (bool): If True, prints progress information.
    
    Returns:
    - feature_importances (pd.DataFrame): A dataframe containing feature names and their importance.
    """
    # Prepare the train data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Drop unwanted columns
    y_train = train_data['impl_volatility']  # Target variable (implied volatility)

    # Standardize the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Train a RandomForestRegressor model
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    
    if verbose:
        print(f"Training RandomForest model with {n_estimators} trees...")

    rf_model.fit(X_train_scaled, y_train)

    if verbose:
        print("Performing permutation importance...")

    # Perform permutation importance on the training data
    perm_importance = permutation_importance(rf_model, X_train_scaled, y_train, n_repeats=10, random_state=random_state, scoring='r2')

    # Create a DataFrame for feature importances
    feature_importances = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': perm_importance.importances_mean,
        'Importance_std': perm_importance.importances_std
    })

    # Sort by importance
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False).reset_index(drop=True)

    return feature_importances


def plot_elbow_curve(feature_importances):
    """
    Plot an elbow curve to determine how many features to select based on permutation importance.
    
    Parameters:
    - feature_importances (pd.DataFrame): A dataframe containing feature names and their importance.
    """
    # Cumulative importance to capture the "elbow"
    cumulative_importance = np.cumsum(feature_importances['Importance'])

    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_importance) + 1), cumulative_importance, marker='o', linestyle='--')
    plt.title('Elbow Plot of Feature Importance')
    plt.xlabel('Number of Features')
    plt.ylabel('Cumulative Importance')
    plt.grid(True)
    plt.show()


def select_top_features(feature_importances, threshold=0.85):
    """
    Select top features based on a cumulative importance threshold.
    
    Parameters:
    - feature_importances (pd.DataFrame): A dataframe containing feature names and their importance.
    - threshold (float): Cumulative importance threshold (e.g., 0.85 for 85%).
    
    Returns:
    - selected_features (list): A list of top features to use in the model.
    """
    cumulative_importance = np.cumsum(feature_importances['Importance'])
    
    # Find how many features explain the threshold (e.g., 85%) of importance
    num_features = np.argmax(cumulative_importance >= threshold) + 1
    
    selected_features = feature_importances['Feature'].iloc[:num_features].tolist()
    print(f"\nSelected {num_features} features based on an {threshold*100}% importance threshold.")
    
    return selected_features


# Example usage for tot_data_train_c
print("Evaluating features for Call options...")
feature_importances_call = feature_permutation_importance(tot_data_train_c, verbose=True)

# Display top features and their importance
print("\nFeature importances for Call options:")
print(feature_importances_call)

# Plot the elbow curve to decide on the number of features
plot_elbow_curve(feature_importances_call)

# Select top features (e.g., 85% cumulative importance)
selected_features_call = select_top_features(feature_importances_call, threshold=0.85)

print("\nSelected top features for Call options:")
print(selected_features_call)

# Example usage for tot_data_train_p
print("\nEvaluating features for Put options...")
feature_importances_put = feature_permutation_importance(tot_data_train_p, verbose=True)

# Display top features and their importance
print("\nFeature importances for Put options:")
print(feature_importances_put)

# Plot the elbow curve to decide on the number of features
plot_elbow_curve(feature_importances_put)

# Select top features (e.g., 85% cumulative importance)
selected_features_put = select_top_features(feature_importances_put, threshold=0.85)

print("\nSelected top features for Put options:")
print(selected_features_put)


Evaluating features for Call options...
Training RandomForest model with 100 trees...
Performing permutation importance...


KeyboardInterrupt: 

In [19]:
feature_importances_call


Feature  Importance  Importance_std
0              prev_day_iv    1.172559    5.026301e-03
1                  idiovol    0.058042    8.324372e-04
2                        T    0.047073    1.031986e-03
3    trading_days_till_exp    0.044550    9.253842e-04
4             prev2_day_iv    0.036632    5.130940e-04
..                     ...         ...             ...
117                     dy    0.000010    1.950613e-07
118             securedind    0.000005    2.400097e-07
119                   sic2    0.000004    4.783520e-07
120                     rd    0.000001    9.117841e-08
121                convind    0.000001    2.172418e-07

[122 rows x 3 columns]

In [21]:
# Get the top 9 most important features
top_9_features_c = feature_importances_call['Feature'].head(8).tolist()

# Print the result
print("top_features_c =", top_9_features_c)


top_features_c = ['prev_day_iv', 'idiovol', 'T', 'trading_days_till_exp', 'prev2_day_iv', 'moneyness', '1Y_bond', 'vol_stock']


In [15]:
feature_importances_put

Feature  Importance  Importance_std
0              prev_day_iv    1.298883    6.623502e-03
1                        T    0.063160    9.296492e-04
2    trading_days_till_exp    0.042703    4.014707e-04
3             prev2_day_iv    0.039644    2.209615e-04
4                  idiovol    0.038580    7.264841e-04
..                     ...         ...             ...
117                     dy    0.000010    5.916638e-07
118             securedind    0.000010    6.587069e-07
119                   sic2    0.000008    3.921294e-07
120                convind    0.000003    2.718937e-07
121                     rd    0.000001    8.694506e-08

[122 rows x 3 columns]

In [22]:
# Get the top 9 most important features
top_9_features_p = feature_importances_put['Feature'].head(7).tolist()

# Print the result
print("top_features_p =", top_9_features_p)


top_features_p ['prev_day_iv', 'T', 'trading_days_till_exp', 'prev2_day_iv', 'idiovol', 'moneyness', 'vol_stock']


In [1]:
feature_importances_call

NameError: name 'feature_importances_call' is not defined